## What We've Learned So Far ✅

1. **Why do we need function calling?** 🤔
2. **How to create and invoke function calls** 🛠️
3. **How to structure LLM output** 📜
4. **Caching LLM output for speed and cost efficiency** 💰⚡
5. **How to parameterize model parameters using configurables** 🔧

If you haven't learnt these things, please go through this notebook again [001-chat-models.ipynb](001-chat-models.ipynb)


---
# 🚀 Exploring New Concepts in LLMs

Now, let's focus on some new things here.  

## 1️⃣ Structuring Input for LLMs  

- We already know how to structure **LLM model responses**, but how should we **structure the input** sent to an LLM?  
- What are the **benefits** of structuring inputs properly?  
- We'll explore these questions and understand the impact of well-structured inputs.  

## 2️⃣ Differentiating Messages in LLM Interactions  

- When we send a message to an LLM, it **replies back** with a new message.  
- We provide **instructions** to the LLM as messages.  
- However, we are using the same term, **"messages,"** for everything, which can cause confusion. 🤔  
- To solve this, **LangChain** provides a better way to **differentiate these messages**—we'll look into how it works.  

## 3️⃣ Understanding Context Window Limitations  

- LLM models have something called a **context window** 📏, meaning they can handle only a **specific number of tokens** (aka words) per request.  
- This limitation is a **challenge** because we want the LLM to **retain more context** for better responses.  
- Fortunately, **LangChain offers a solution** to this problem—we'll explore that too!  

Stay tuned as we dive deeper into these concepts! 🔍✨  



### Let's Start with Message Types

1. **SystemMessage**: We use this to instruct LLM on what it has to do
2. **HumanMessage**: We use this to communicate to LLM (input to LLM)
3. **AIMessage**: llm message will be of this type
4. **ToolMessage**: We will take out this bit later, but this message type is used to store `tool output`

In [1]:
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage,ToolMessage

In [2]:
human_input = HumanMessage(content="Hello GPT. i would like to know weather details in my location", # content is used to store the message itself
                          id="123", # id must be unique for all messages. we identify a unique message only with this unique id
                          name="alex", # a top-level filter on id. So when we want to filter messages, we start with name so that messages will be from specific entity and id to get specific id
                          custom_metadata={ # this is custom metadata
                              "phone_no":"+1867389126",
                              "location":"santa clara"
                          })

In [4]:
human_input.content

'Hello GPT. i would like to know weather details in my location'

In [9]:
# content, name and id are common arguments are there for system_message too
system_message = SystemMessage(content="You are dolphin a helpful ai assistant",id="321",name="persona")

In [26]:
# AIMessage and ToolMessage have more attributes to play with

ai_message = AIMessage(content="hello! i am dolphin your ai assistant. what do you want to know?",
                      name="model",
                      id="604880nv4",
                      tool_calls=[{ # additional param to store tool_calls
                          "name":"multiplier", # example. i didnt define this tool
                          "args":{"a":40,"b":50},
                          "id":"ADTG3793Y9", # this id is a combination of toolname and it's uniqueness. I mean if same tool is invoked twice. this value will be different
                          "type":"tool_call"
                      }])

In [27]:
ai_message.tool_calls

[{'name': 'multiplier',
  'args': {'a': 40, 'b': 50},
  'id': 'ADTG3793Y9',
  'type': 'tool_call'}]

In [28]:
tool_message = ToolMessage(content="2000",
                           name="multiplier_tool_result",
                           id="fvi3r792bc22", # this is unique id for the message
                          tool_call_id="ADTG3793Y9",# this is an example, id for tool mentioned above and this will be same
                           artifact={"log":"result is an even number"}, # sometimes tool might return some metadat which are required but not used by model. these can be stored here
                           status="success", # this is a status Literal message with either success or error in it
                          )

In [29]:
tool_message

ToolMessage(content='2000', name='multiplier_tool_result', id='fvi3r792bc22', tool_call_id='ADTG3793Y9', artifact={'log': 'result is an even number'})

In [197]:
## we can invoke our LLM with these message types
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv

load_dotenv()

llm = init_chat_model("deepseek-r1-distill-llama-70b",model_provider="groq",max_tokens=1000)

In [46]:
resp = llm.invoke([system_message,human_input]) # this is how you can invoke your llm model

In [50]:
user_input = system_message + human_input # yes we can add messages

In [52]:
type(user_input) # This is our next topic. we will explore this

langchain_core.prompts.chat.ChatPromptTemplate

In [56]:
user_input.messages # you can see both messages in a list here

[SystemMessage(content='You are dolphin a helpful ai assistant', additional_kwargs={}, response_metadata={}, name='persona', id='321'),
 HumanMessage(content='Hello GPT. i would like to know weather details in my location', additional_kwargs={}, response_metadata={}, name='alex', id='123', custom_metadata={'phone_no': '+1867389126', 'location': 'santa clara'})]

In [71]:
## there is one more think called Chunk, this chunk exist in all above message types
from langchain_core.messages import HumanMessageChunk,AIMessageChunk

hm1 = HumanMessageChunk(content="I am using a desktop computer for this tutorial.") # same arguments as in normal HumanMessage
hm2 = HumanMessageChunk(content="I am noting down all the points i have discussed and about to discuss.")

In [72]:
hm1 + hm2 # Add two chunks, you will get a new chunk

HumanMessageChunk(content='I am using a desktop computer for this tutorial.I am noting down all the points i have discussed and about to discuss.', additional_kwargs={}, response_metadata={})

In [73]:
human_input + human_input # But add two same or different messages, you will get a chatprompt template

ChatPromptTemplate(input_variables=[], input_types={}, partial_variables={}, messages=[HumanMessage(content='Hello GPT. i would like to know weather details in my location', additional_kwargs={}, response_metadata={}, name='alex', id='123', custom_metadata={'phone_no': '+1867389126', 'location': 'santa clara'}), HumanMessage(content='Hello GPT. i would like to know weather details in my location', additional_kwargs={}, response_metadata={}, name='alex', id='123', custom_metadata={'phone_no': '+1867389126', 'location': 'santa clara'})])

## Now Let's Structure our inputs

In [143]:
# There are many ways we can prompt the model

# 1. simple prompt (string)
llm.invoke("what is your name")

AIMessage(content="<think>\n\n</think>\n\nGreetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 7, 'total_tokens': 51, 'completion_time': 0.16, 'prompt_time': 0.003494816, 'queue_time': 0.023829819, 'total_time': 0.163494816}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_af2982c95d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2aabfaec-7d1e-466c-8588-eb34baf5ff0a-0', usage_metadata={'input_tokens': 7, 'output_tokens': 44, 'total_tokens': 51})

In [144]:
# 2. simple prompt (string)
topic = "fiance"
llm.invoke(f"""
Please get me only short responses (50 words) for the questions i am going to ask.

tell me a joke on {topic}
""")

AIMessage(content='<think>\nOkay, so the user wants a joke about a fiancé, and they want the response to be short, around 50 words. Let me brainstorm some ideas. Maybe something playful about marriage or engagement. I should make sure it\'s lighthearted and not offensive. Let\'s think about common fiancé stereotypes or situations. Oh, maybe something about love and commitment. How about a play on words? Maybe something like, "Why did the fiancé bring a ladder to the wedding? Because they wanted to take their relationship to new heights!" Hmm, that\'s a bit cheesy but works. Alternatively, I could go with something about the ring or planning the wedding. Wait, the user already provided an example, so I can use that as a model. Their example is about love being blind and marriage opening the eyes, which is a classic twist. Maybe I can think of another angle. Perhaps something about the fiancé forgetting something, like "Why did the fiancé forget his keys? Because he was too excited to un

In [145]:
#3. chat prompt (json openai format)
llm.invoke([
    {"role":"system","content":"please make sure to return your response in 10 words"},
    {"role":"user","content":"tell me a joke about iron man in marvel"}
])

AIMessage(content='<think>\nAlright, so the user is asking for a joke about Iron Man from the Marvel universe. They also specified that the response should be in 10 words. Hmm, I need to come up with something that\'s both funny and concise. \n\nFirst, I should think about Iron Man\'s key traits. He\'s a billionaire, a genius inventor, and of course, he wears that iconic red and gold suit. Maybe I can play on words related to his suit or his personality. \n\nI remember that Iron Man\'s suit is powered by an arc reactor, but that might be a bit too technical for a joke. Maybe something about his wealth? Like, why did Iron Man bring a ladder to the party? Because he heard the drinks were on the house! That plays on the phrase "on the house," meaning free drinks, but also references the height, which a ladder would help with. Plus, it\'s within the 10-word limit. \n\nWait, let me check the word count. "Why did Iron Man bring a ladder to the party? Because he heard the drinks were on the h

In [147]:
#4. chat prompt (tuples)
llm.invoke([
    ("system","please make sure to return your response in 10 words"),
    ("user","tell me a joke about iron man in marvel")
])

AIMessage(content='<think>\nOkay, so the user wants a joke about Iron Man from Marvel. Let me think... Iron Man\'s suit is a big part of his character, so maybe I can make a pun around that. Why did Iron Man\'s suit go to therapy? Hmm, because it had a lot of "metal" issues. That plays on "metal" as in the suit\'s material and "mental" issues, which is a common pun. It\'s simple and should bring a smile. I think that works.\n</think>\n\nWhy did Iron Man\'s suit go to therapy? It had too many *metal* issues.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 125, 'prompt_tokens': 24, 'total_tokens': 149, 'completion_time': 0.454545455, 'prompt_time': 0.0056447, 'queue_time': 0.096018251, 'total_time': 0.460190155}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_95405939f8', 'finish_reason': 'stop', 'logprobs': None}, id='run-39421864-af81-4bc3-a89a-57caf2d3d353-0', usage_metadata={'input_tokens': 24, 'output_tokens': 125, 'total_tok

In [148]:
#5. chat prompt (MessageTypes)
llm.invoke([
    SystemMessage(content="please make sure to return your response in 10 words"),
    HumanMessage(content="tell me a joke about iron man in marvel")
])

AIMessage(content='<think>\nOkay, so I need to come up with a joke about Iron Man from Marvel. Let me think about Iron Man\'s characteristics. He\'s a superhero with a suit made of iron, right? Oh, wait, no, it\'s not actually iron, it\'s more advanced metals and technology. But "iron" is in his name, so maybe that\'s a pun opportunity.\n\nI remember that in the Marvel movies, Tony Stark, who is Iron Man, is a bit of a jokester himself. Maybe I can play on that. Let me think about things related to iron. Iron is a metal, and in everyday language, people might say things like "ironing out the wrinkles" or "ironing clothes." That could be a good angle.\n\nSo, if Iron Man were to make a joke, maybe it would involve something he does. Since he\'s always suited up and saving the day, perhaps he could joke about something mundane. Maybe he could say something about not needing to iron his suit because it\'s always perfect. Or maybe something about how he doesn\'t have to iron clothes because

### Now Let's parameterize some of these

In [151]:
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,MessagesPlaceholder

# For simpler string prompts
prompt = PromptTemplate.from_template("Hi! My name is {person}")
prompt.invoke({"person":"alex"}) # we can see even prompt can be invokable not only model or tool

StringPromptValue(text='Hi! My name is alex')

What if we want to add many **few-shot examples** to the **string prompt**. we dont want to create one big repeated string

1. we need a format for each example. so first, we need to create template for this.
2. now we need to have a list of all few-shot examples
3. with above two points, langchain will create a bigstring for us. now we want to add prefix and suffix for that string.

In [200]:
from langchain_core.prompts import FewShotPromptTemplate,PromptTemplate

few_shot_template = PromptTemplate.from_template("user: {user}\nai: {ai}")
examples = [
    {"user":"Add 5 and 2","ai":"Addition: 7"},
    {"user":"Multiply 10 and 5","ai":"Multiplication: 50"}
]
final_template = FewShotPromptTemplate(example_prompt=few_shot_template,
                                       examples=examples,
                                       prefix="Observe Below example conversations and answer accordingly",
                                       suffix="user: {user}\n ai: ",
                                       input_variables=["user"]
                                      )

template_value = final_template.invoke({"user":"Add 10 and 7"})

In [199]:
print(template_value.text) # You can see below prefix, examples and suffix are added.

Observe Below example conversations and answer accordingly

user: Add 5 and 2
ai: Addition: 7

user: Multiply 10 and 5
ai: Multiplication: 50

user: Add 10 and 7
 ai: 


In [160]:
llm.invoke(template_value) # invoked with llm

AIMessage(content='<think>\nI need to analyze the user\'s request based on the provided examples.\n\nIn the examples, the user asks for a calculation, and the AI responds with the operation and the result.\n\nThe user\'s current query is "Add 10 and 7".\n\nFollowing the pattern, I should respond with "Addition: 17".\n</think>\n\nAddition: 17', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 57, 'total_tokens': 132, 'completion_time': 0.272727273, 'prompt_time': 0.010422477, 'queue_time': 0.259157912, 'total_time': 0.28314975}, 'model_name': 'deepseek-r1-distill-llama-70b', 'system_fingerprint': 'fp_af2982c95d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0967acd3-186d-4ee3-92b4-c29f21106d13-0', usage_metadata={'input_tokens': 57, 'output_tokens': 75, 'total_tokens': 132})

In [172]:
# llm.invoke(final_template.invoke({"user":"Add 10 and 7"})) # instead of doing this, we can use LCEL. we will go through this later deeply

# LCEL syntax aka chains
new_llm = final_template | llm # invoked params first goes through final_template and output which is stringprompt aka normalstring is passed to llm
new_llm.invoke({"user":"Add 10 and 7"})

AIMessage(content='<think>\nOkay, so I need to figure out how to respond to the user\'s instruction based on the examples provided. Let\'s see, the user has given a pattern where they ask for a mathematical operation between two numbers, and the AI responds with the operation\'s name followed by the result.\n\nLooking at the first example: the user says "Add 5 and 2", and the AI responds with "Addition: 7". That makes sense because 5 plus 2 equals 7. \n\nThen, the second example is "Multiply 10 and 5", and the AI says "Multiplication: 50". Again, that\'s correct because 10 multiplied by 5 is 50.\n\nNow, the third example is "Add 10 and 7", and the AI\'s response is cut off. Following the pattern, the AI should respond with "Addition: 17" because 10 plus 7 is 17.\n\nSo, the user is probably testing if I can follow the same pattern. They might be checking if I can recognize the operation and compute the result correctly. Maybe they\'re evaluating my ability to handle basic arithmetic ope

In [173]:
# for chat prompt inputs with tuples we will use ChatPromptTemplate
prompt = ChatPromptTemplate([("system","You are a helpful assistant in {topic}"),
                             ("user","I need your help in {task}")
                            ])
prompt.invoke({"topic":"Finance","task":"Doing my taxes"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant in Finance', additional_kwargs={}, response_metadata={}), HumanMessage(content='I need your help in Doing my taxes', additional_kwargs={}, response_metadata={})])

In [174]:
# for chat prompt inputs with openai like request we will use ChatPromptTemplate
openai_prompt = ChatPromptTemplate([
    {"role":"system","content":"You are a helpful assistant in {topic}"},
    {"role":"user","content":"I need your help in {task}"}])

openai_prompt.invoke({"topic":"Lawsuits","task":"filing a cases againt racoons. They are invading my garden."})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant in Lawsuits', additional_kwargs={}, response_metadata={}), HumanMessage(content='I need your help in filing a cases againt racoons. They are invading my garden.', additional_kwargs={}, response_metadata={})])

In [183]:
# for chat prompt inputs with langchain messagetypes we will use ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate

openai_prompt = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template("You are a helpful assistant in {topic}"),
    HumanMessagePromptTemplate.from_template("I need your help in {task}")])

openai_prompt.invoke({"topic":"financial advising","task":"suggesting me whether i can afford jeep wrangler rubicon?"})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant in financial advising', additional_kwargs={}, response_metadata={}), HumanMessage(content='I need your help in suggesting me whether i can afford jeep wrangler rubicon?', additional_kwargs={}, response_metadata={})])

In [134]:
response = llm.invoke(prompt.invoke({"topic":"Finance","task":"Doing my taxes"})) 

In [135]:
response.content

"<think>\nOkay, so I need to do my taxes, but I'm not really sure where to start. Let me try to break this down step by step. First, I remember from last year that I needed some forms from my employer. I think it's called a W-2? Yeah, that sounds right. I should probably check my emails or maybe my employer's portal to see if that's available yet. If I can't find it, I guess I'll have to contact HR or my HR department. \n\nNext, I think I need to report any other income I had. I did some freelance work over the summer, so I should get a 1099 form for that. Wait, do I get a 1099 every time I freelance, or is it only if I made a certain amount? I think it's if I made over $600, but I'm not entirely sure. Maybe I should look that up to confirm. Also, I sold some stuff online, like on eBay. I'm not sure if that counts as income or just personal stuff. I think if it's personal, it's not taxable, but if I was selling things regularly, maybe it's considered business income. I'm a bit confused

In [136]:
prompt = ChatPromptTemplate([
    ("system","You are a helpful assistant"),
    MessagesPlaceholder("user_inputs") # you can mention a placeholder and you can invoke in prompt
])
prompt.invoke({"user_inputs":[
    ("user","you know my name?"),
    ("ai","Sorry you haven't told me before. You can say it now,i will remember"),
    ("user","my name is rahul")
]})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='you know my name?', additional_kwargs={}, response_metadata={}), AIMessage(content="Sorry you haven't told me before. You can say it now,i will remember", additional_kwargs={}, response_metadata={}), HumanMessage(content='my name is rahul', additional_kwargs={}, response_metadata={})])

In [189]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

messages = [
    {"user": "multiply 5 with 10","ai":"Multiplication: 50"},
    {"user": "add 34 with 64", "ai":"Addition: 98"}
]

fewshot_message_template = ChatPromptTemplate([
    ("user","{user}"),
    ("ai","{ai}")
])

fewshot_temp = FewShotChatMessagePromptTemplate(example_prompt=fewshot_message_template,
                                               examples=messages)

In [193]:
fewshot_temp.invoke({}) # sometimes it is just few-shot we dont need to pass other values.so we used empty {} in invoke

ChatPromptValue(messages=[HumanMessage(content='multiply 5 with 10', additional_kwargs={}, response_metadata={}), AIMessage(content='Multiplication: 50', additional_kwargs={}, response_metadata={}), HumanMessage(content='add 34 with 64', additional_kwargs={}, response_metadata={}), AIMessage(content='Addition: 98', additional_kwargs={}, response_metadata={})])

In [194]:
final_prompt = ChatPromptTemplate([
    ("system","You are a helpful ai assistant"),
    fewshot_temp,
    ("user","add 6 with 6")
])

special_chainer = final_prompt | llm

In [195]:
special_chainer.invoke({})

AIMessage(content='<think>\nAlright, so I\'ve got this math problem here: "add 6 with 6." Okay, that seems pretty straightforward, but since I\'m just starting out with this whole math thing, I want to make sure I understand it completely. Maybe I should break it down step by step to really grasp what\'s going on.\n\nFirst off, what does it mean to "add" two numbers? I think adding is like combining two quantities together to get a total. So if I have 6 apples and someone gives me another 6 apples, how many apples do I have in total? That\'s basically what addition is, right? It\'s putting things together to find out how much you have altogether.\n\nOkay, so in this case, I have the number 6, and I need to add another 6 to it. Let me write that down to visualize it better: 6 + 6. Now, I need to figure out what 6 plus 6 equals. I remember from school that addition is one of the basic operations, and it\'s pretty fundamental. But I want to make sure I\'m not just memorizing the answer wi

# Time to Recap: LangChain Message Types & Structured Inputs

## 📌 Message Types

1. We have explored different message types in LangChain.
2. Commonly used attributes in these message types are:
   - `content`
   - `name`
   - `id`
3. `AIMessage` has one more important attribute: `tool_calls` (used to call tools).
4. `ToolMessage` has three additional important attributes:
   - `tool_call_id`
   - `artifact`
   - `status`
5. There is a concept called **Chunks** for all message types. These come into play when streaming LLM output.
6. We can:
   - Combine two message chunks to create another chunk.
   - Add two or more `MessageTypes` to create a `ChatPromptTemplate`.

## 🏗️ Structured Inputs

1. There are mainly two types of input to LLM:
   - **Prompt** (string)
   - **Chat prompt list** (messages)
2. A prompt can be either a normal string or a docstring.
3. Chat prompts can be of three types:
   1. List of dictionaries (OpenAI style)
   2. List of tuples
   3. List of LangChain message types
4. Just like models and tools, even a prompt is **invokable**.
5. Since there are two types of inputs to LLM, at a high level, there are two types of templates:
   - `PromptTemplate`
   - `ChatPromptTemplate`
6. `PromptTemplate` supports only strings, while `ChatPromptTemplate` supports both:
   - List of dictionaries
   - List of tuples
7. Specific message type templates like `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate`, etc., should be used in `ChatPromptTemplate` for LangChain message types.
8. We can extend **few-shot examples** using respective `PromptTemplates`:
   - For a **few-shot prompt template**, we need:
     - `examples`
     - `example_prompt` (which is a `PromptTemplate`)
   - For a **few-shot chat prompt template**, we need:
     - `examples`
     - `example_prompt` (which is a `ChatPromptTemplate`)
9. When we talk about **configurable prompts**, we can control:
   - Specific parts in a message
   - Adding multiple messages to `ChatPromptTemplate` as a placeholder using `MessagePlaceholder` (only works for chat prompt messages)
10. With structured inputs, it is better to use **LCEL syntax** for better code. Otherwise, we have to use `invoke` for passing the prompt to the template and for the model too.

🚀 **practicing a lot to master these concepts!**

